In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import json
import csv
from pandas import DataFrame
import time
import gc

from IPython.display import Image
from IPython.core.display import HTML
from sklearn import linear_model
from sklearn.metrics import accuracy_score

from scipy.sparse import csr_matrix

%matplotlib inline 

import os
print(os.listdir("."))

['.git', '.gitignore', '.ipynb_checkpoints', 'binary_classifier.ipynb', 'Classification.ipynb', 'data', 'feature_extractor.py', 'feature_extractor_linux.py', 'keras_test.py', 'LICENSE', 'mlp_200_100.pkl', 'multilabel one-vs-all classification.ipynb', 'README.md', 'rescale.py', 'Silllina.ipynb', 'submission.csv', 'Untitled.ipynb']


In [2]:
import glob
import pickle

FEATURES_LOCATION = './data/features/'
F_CORE = 'cnn_features_'


def get_label_from_path(file):
    return file.split('\\')[1].split('.')[0]

def load_data(mode):
    if(mode == 'test'):
        pickle_path = F_CORE + mode
        data = pickle.load(open(FEATURES_LOCATION + pickle_path + '.pkl', 'rb'))
        to_return = {}
        for key, value in list(data.items()):
            to_return[get_label_from_path(key)] = value.reshape(1,-1)
        return to_return, None
    
    pickle_path = F_CORE + mode + '_'
    
    data = {}
    for i in range(1,129):
        data[i] = pickle.load(open(FEATURES_LOCATION + pickle_path + str(i) + '.pkl', 'rb'))
        
    X = []
    y = []
    for key, value in list(data.items()):
        #change all labels that is not the chosen label to 0, and change the chosen label to 1
        the_class = key
        features = np.array(list(value.values()))
        for feature in features:
            y.append(the_class)
            X.append(feature)

    return np.array(X), np.array(y)

In [3]:
#TODO: use perceptron instead of logreg, use subset of data

X_train, y_train = load_data('train')
print("Xtrain.shape: ",X_train.shape)
print("ytrain.shape: ",y_train.shape)
# load the training data and rewrite the classes
X_val, y_val = load_data('valid')
print("X_val.shape: ",X_val.shape)
print("y_val.shape: ",y_val.shape)

Xtrain.shape:  (192171, 2048)
ytrain.shape:  (192171,)
X_val.shape:  (6309, 2048)
y_val.shape:  (6309,)


In [8]:
NUM_CLASSES_TO_TEST = 129 # number of classes to do binary classification on, 129 = max
score_mat = np.zeros([X_val.shape[0],129])

for i in range(1,NUM_CLASSES_TO_TEST):
    class_i_X_train = np.copy(X_train)
    class_i_y_train = np.copy(y_train)
    class_i_X_val = np.copy(X_val)
    class_i_y_val = np.copy(y_val)
    
    # change the trianing data
    for j in range(0,len(class_i_y_train)):
        if class_i_y_train[j] != i:
            class_i_y_train[j] = 0
        else:
            class_i_y_train[j] = 1
        
    # change the validation data
    for k in range(0,len(class_i_y_val)):
        if class_i_y_val[k] != i:
            
            class_i_y_val[k] = 0
        else:
            class_i_y_val[k] = 1
            
    '''  
    logreg = linear_model.LogisticRegression()
    # we create an instance of Neighbours Classifier and fit the data.
    logreg.fit(class_i_X_train, class_i_y_train)
    print("training complete")
    score = logreg.score(class_i_X_val,class_i_y_val)
    score_list.append(score)
    print("accuracy: ",score)
    '''
    # save result in score_mat[feature_num, i]
    
    ptron = linear_model.Perceptron()
    ptron.fit(class_i_X_train,class_i_y_train)
    preds = ptron.predict(class_i_X_val)
    
    for p in range(0,len(preds)):
      score_mat[p,i] = preds[p]

print("score matrix completed")   

C:\Users\Tobias\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


score matrix completed


In [43]:
#I now have a matrix of shape: "X_val.shape[0],129" containing the votes
# now make every class "vote" for the label they think it was. 
# 1 = class thinks it was itself, 0 = class thinks it was another class.
#vote_list = np.empty([score_mat.shape[0],1])
vote_list = np.zeros([score_mat.shape[0]])
for i in range(0,score_mat.shape[0]):
    for j in range(0,score_mat.shape[1]):
        # if a class think its the class, then save it in the list
        # this is flawed since it first come first serve.
        if score_mat[i,j] == 1:
            # feature i, is now predicted to be feature j
            vote_list[i] = j
            break
        
# now calculate accuracy
print(y_val.shape)
print(len(vote_list))
accuracy = accuracy_score(vote_list,y_val)
print("accuracy: ", accuracy)


(6309,)
6309
accuracy:  0.5534950071326676
